# Introduzione


Carico il dataset creato da `create_dataset.py` da google drive

> 
L'obbiettivo del progetto è quello di implementare un classificatore che sia in grado di distinguere quando un secchio è pieno, a metà, vuoto. Abbiamo registrato dei video per tre secchi diversi, dove abbiamo ripreso se i secchi fossero vuoti, pieni o a metà. I video sono stati registrati a 30fps in alta risoluzione. (iPhone XS e Huawei P20)

Abbiamo estratto per video 100 frame utilizzando il codice presente sul file `create_dataset.py`

Sono state associate per ogni immagine delle classi numeriche rispettivamente:
- -1: empty
- 0: half
- 1: full

Le label sono state inserite nel file `all_labels.txt` rispettivamente nell'ordine
`'nome-immagine': 'classe-appartenza'`

###### Il file all labels è stato letto da una funzione implementate in create dataset dove viene effettuata una permutazione randomica su tutte le 900 labels e sono state suddivise tra train e test (650 per training 250 test)

**i file .txt li ho importati brutalmente spiegare poi meglio il rpocedimento**

In [7]:
## mount google drive
from google.colab import drive
import zipfile

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
!unzip /content/gdrive/MyDrive/img.zip -d /content/gdrive/MyDrive/dataset/

Archive:  /content/gdrive/MyDrive/img.zip
   creating: /content/gdrive/MyDrive/dataset/img/
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_143.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_625.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_631.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_157.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_619.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_180.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_194.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_802.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_816.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_427.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_341.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_355.jpg  
  inflating: /content/gdrive/MyDrive/dataset/img/trashbean_433.jpg  
  inflating

# Todo
- calcolare media e dev std
- Implementare alexnet v3 in quanto è più performante
- implementare train classifier di lezione 4
- tensorboard per visualizzare i grafi

In [35]:
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from PIL import Image
from os import path
from torchvision import transforms
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
from tqdm import tqdm


from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import random

# Construction of dataset

- `all_labels.txt`: All image's labels of all entire dataset
- `classes.txt`: classes
- `test.txt`: test dataset
- `test.txt`: train dataset 


In [25]:
class TrashbeanDataset(Dataset):
    def __init__(self, base_path, txt_list, transform=None):
        self.base_path = base_path
        self.images = np.loadtxt(txt_list, dtype=str, delimiter=',')
        self.transform = transform

    def __getitem__(self, index):
        f,c = self.images[index] # recuperiamo il path dell'immagine di indice index e la relativa etichetta
        im = Image.open(self.base_path + f)   ## load img with PIL
        
        if self.transform is not None:          ## toTensor, resize, normalize etc...
            im = self.transform(im)

        label = int(c) # converto l'etichetta in intero

        return (im, label)  # usare una tupla sembra l'opzione migliore in termini di velocità e memoria 

    def __len__(self): # restituisce numero di campioni: la lunghezza della lista images
        return len(self.images)

Carichiamo il dataset in esempio per visualizzare che la classe funzioni correttamente

In [47]:
dic = {
    -1: "empty",
    0: "half",
    1: "full"
}

path = 'gdrive/MyDrive/dataset/'

dataset = TrashbeanDataset(path + 'img/', path + 'labels/all_labels_rand.txt', transform=transforms.ToTensor())

# visualizziamo il primo elemento con __geitem e controlliamo che il tipo assegnato sia corretto

print(dataset.__len__())

print(dataset[0][0].size())

## plt.figure(figsize=(15,10))
## for i in range(10):
##     plt.subplot(2,5,i+1)
##     plt.imshow(np.asarray(dataset[i][0]), aspect="auto")
##     plt.xlabel(dic[dataset[i][1]])
## plt.show()


900
torch.Size([3, 1920, 1080])


## Adesso è necessario calcolare media e varianza

Dobbiamo calcolare media e varianza per ogni immagine del dataset intero per ogni canale

(media_r, media_g, media_b)
(dvstd_r, devst_g, devst_b)

In [54]:
dataset_loader = DataLoader(dataset=dataset, shuffle=False, batch_size=150)  # non serve shuffle perché già randomico

In [50]:
dataset[0][0].shape

torch.Size([3, 1920, 1080])

Calcoliamo:
1.   media
2.   dev std




In [52]:
def mean_std(loader):
  channels_sum, channels_squared_sum, num_batches = 0, 0, 0

  for data, _ in loader:
    channels_sum += torch.mean(data, dim=[0,2,3])
    channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
    num_batches += 1
  
  mean = channels_sum/num_batches
  std = (channels_squared_sum/num_batches - mean**2)**0.5

  return mean, std

In [55]:
mean_std(dataset_loader)

(tensor([0.5106, 0.4575, 0.4110]), tensor([0.2321, 0.2178, 0.2296]))

Secondo l'intero dataset avremo:

- media sarà: **[0.5106, 0.4575, 0.4110]**

- deviazione standard sarà: **[0.2321, 0.2178, 0.2296]**

In [ ]:
transform = transforms.ToTensor()

dataset_train = TrashbeanDataset('static/datasets/img', 'static/datasets/train.txt', transform=transform) # can put (w,h) too#
#dataset_test = TrashbeanDataset('static/datasets/img', 'static/datasets/test.txt', transform=transform) # can put (w,h) too

#print("dim totale dataset train", dataset_train.__len__())
#print("img 1", dataset_train[0]['image'].shape, dataset_train[0]['label'])

Dobbiamo
- Il tipo di modello (??)
- test classifier
- train classifier
- caricare dataset test e training con dataLoader (normalizzati)
- procedura di addestramento